# Práctico 1

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

## Importaciones

In [2]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm, trange

## Constantes

In [28]:
ARCHIVO_SET_DE_ENTRENAMIENTO = './data/meli-challenge-2019/spanish.train.jsonl.gz'
ARCHIVO_SET_DE_PRUEBA = './data/meli-challenge-2019/spanish.test.jsonl.gz'
ARCHIVO_SET_DE_VALIDACION = './data/meli-challenge-2019/spanish.validation.jsonl.gz'
ARCHIVO_TOKENS = './data/meli-challenge-2019/spanish_token_to_index.json.gz'
# ARCHIVO_DE_EMBEDDINGS = './data/SBW-vectors-300-min5.txt.bz2'
ARCHIVO_DE_EMBEDDINGS = '../data/glove.6B.50d.txt.gz'
EPOCHS = 1

## Carga de datos

Esta carga de datos se realiza con el fin de explorar los mismos. Otra carga de datos tendrá lugar al comento de construir el dataset.

In [4]:
%%time
file_paths = [ARCHIVO_SET_DE_ENTRENAMIENTO,
              ARCHIVO_SET_DE_PRUEBA,
              ARCHIVO_SET_DE_VALIDACION]
i = 2 # Hasta que funcione
df = pd.read_json(path_or_buf=file_paths[i], lines=True)

CPU times: user 18.6 s, sys: 2.65 s, total: 21.3 s
Wall time: 22.2 s


## Análisis y visualización de los datos

El **set de entrenamiento** original tiene 4895280 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 4508043 registros no confiables y 387237 registros confiables.
- **title**: El título que se asignó al producto. **Esta información es la que se utilizará para armar el dataser de entrenamiento.**
- **category**: La categoría que se asignó al producto. **Este es el target**.
- **split**: El tipo de dataset. _train_ para el set de entrenamiento.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de entrenamiento.
- **size**: La cantidad de registros. 4895280 registros para el caso del set de entrenamiento.

El **set de prueba** original tiene 63680 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Todas las etiquetas de este dataset son confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _test_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de prueba.
- **size**: La cantidad de registros. 63680 registros para el caso del set de prueba.

El **set de validación** original tiene 1223820 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 1127189 registros no confiables y 96631 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _validation_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de validación.
- **size**: La cantidad de registros. 1223820 registros para el caso del set de validación.

El archivo **spanish_token_to_index** tiene las 50002 correspondencias que existen entre las palabras tokenizadas del título y las etiquetas numéricas bajo la columna data en los sets de entrenamiento, prueba y validación. No se utilizará este tokenizador, en lugar de ello se utilizará ...

**En este trabajo práctico se utiliza:**
- El **set de entrenamiento** para entrenar el modelo
- El **set de validación** para evaluar el modelo y ajustar hiperparámetros
- El **set de prueba** para mostrar el mejor modelo obtenido

In [5]:
df.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,unreliable,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS,validation,"[metal, biela, hdi]","[457, 1480, 3450]",88,632,1223820
1,spanish,unreliable,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS,validation,"[repuestos, martillo, rotoprcutor, bosch, gshs...","[3119, 892, 1, 767, 1, 9337]",174,632,1223820
2,spanish,unreliable,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS,validation,"[pesca, caña, pejerrey, colony, brava, fibra, ...","[700, 990, 2057, 3990, 3670, 1737, 1153, 6568]",313,632,1223820
3,spanish,unreliable,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES,validation,"[porcelanato, abitare, ceramica, portinari]","[2722, 4404, 1406, 4405]",427,632,1223820
4,spanish,unreliable,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS,validation,"[reconstruction, semi, lino, alfaparf, shampoo]","[1, 3365, 7502, 10919, 849]",194,632,1223820


Para el presente trabajo práctico solamente interesan las columnas **title** y **category**.

In [6]:
df.drop(columns=['language', 'label_quality', 'split', 'tokenized_title', 'data', 'target', 'n_labels', 'size'],
        inplace=True)
df.head()

,title,category
0,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS
1,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS
2,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS
3,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES
4,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS


In [16]:
nom_archivo = './data/validation_set.csv'
nom_archivo

'./data/validation_set.csv'

In [17]:
cols = df.columns
cols

Index(['title', 'category'], dtype='object')

In [18]:
df.to_csv(nom_archivo, header=list(cols), index=False)

In [7]:
# En caso de quererlo, se reduce la muestra a n items.
# Esta reducción es útil durante la confección del notebook, para ahorrar tiempo.
n = 1000000
df = df.sample(n, axis=0)

### Etiquetar el target.

A cada una de las 632 categorías del target se le asigna un valor entero entre 0 y 631. La relación entre la categoría u su etiqueta queda almacenada en un dataframe de Pandas llamado `df_categories`.

In [ ]:
categories = df.category.unique()
sorted_categories = np.sort(categories)
df_categories = pd.DataFrame(sorted_categories, columns=['categories'])

In [ ]:
df_categories['cat_tag'] = pd.DataFrame(list(range(df_categories.shape[0])))

In [ ]:
df_categories.set_index('categories', inplace=True)
df_categories.head()

## Construcción del Dataset

El dataset se construye a partir de un dataframe de Pandas que debe tener al menos dos columnas:
- **title**
- **category**

In [8]:
class MeLiChallengeDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, item):
        item = {
            "data": self.df.iloc[item]["title"],
            "target": self.df.iloc[item]["category"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

## Preprocesamiento de los datos

El preprocesamiento de texto tiene dos propósitos:
- Tokenizar los títulos (datos) de modo que se quiten los signos de puntuación y palabras cortas como preposiciones y conjunciones (stopwords), todas las palabras queden en minúsculas, se separen en listas de palabras, etc.
- Transformar las categorías en etiquetas numéricas.

In [9]:
class RawDataProcessor:
    def __init__(self, dataset, ignore_header=True, vocab_size=50000):
        self.filters = [lambda s: s.lower(),
                        preprocessing.strip_tags,
                        preprocessing.strip_punctuation,
                        preprocessing.strip_multiple_whitespaces,
                        preprocessing.strip_numeric,
                        preprocessing.remove_stopwords,
                        preprocessing.strip_short]
        
        # Esta clase encapsula el mapeo entre las palabras normalizadas y sus correspondientes indices 
        # https://radimrehurek.com/gensim/corpora/dictionary.html
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist()
        )
        
        # Filter the dictionary with extremos words
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html?highlight=filter_extrem
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        
        # Asigna nuevos índices a todas las palabras
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.compactify.html
        # https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.compactify
        self.dictionary.compactify()
        
        # Se agregan tokens especiales
        self.dictionary.patch_with_special_tokens({"[PAD]": 0,
                                                   "[UNK]": 1})
        
        # Conversión de categorías a etiquetas
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}


    def _preprocess_string(self, string):
        # Procesamiento de los títulos mediante la aplicación de una lista de filtros
        # Parámetro: str -> El título sin procesar
        # Salida: list -> Lista de strings
        # https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        # Convierte una lista de palabras en una lista de índices
        # Parámetro: list -> Lista de palabras
        # Salida: list -> Lista de enteros (índices) en el mismo orden que las palabras
        # https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        # Convierte un string en una lista de índices
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        # Convierte las categorías a etiquetas
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

In [10]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

In [11]:
processor = RawDataProcessor(df)
dataset = MeLiChallengeDataset(df, transform=processor)

In [12]:
i = 1000
print(f"El dataset tiene {len(dataset)} elementos.")
print(f"Elemento #{i}:\n\tData: {dataset[i]['data']}\n\tTarget: {dataset[i]['target']}")

El dataset tiene 1000000 elementos.
Elemento #1000:
	Data: [1178, 3084, 3085, 3080, 720, 3086]
	Target: 297


## Carga del Dataset

In [13]:
pad_sequences = PadSequences()
train_loader = DataLoader(dataset,
                          batch_size=100,
                          shuffle=True,
                          collate_fn=pad_sequences,
                          drop_last=False)

## Construcción del Modelo

In [24]:
class MeLiChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 632)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

## Algoritmo de Optimización

In [25]:
model = MeLiChallengeClassifier(ARCHIVO_DE_EMBEDDINGS, processor.dictionary, 50, True)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Entrenamiento del modelo

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [40]:
%%time
for epoch in range(EPOCHS):  # Recorre el dataset multiples veces
    model.train()
    running_loss = 0.0
    for data in train_loader:
        inputs = data['data'].to(device)
        labels = data['target'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels.squeeze().long())
        loss.backward()
        optimizer.step()

## Evaluación del Modelo

## Guardado de los parámetros del modelo